In [4]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')

In [5]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 125kB/s  eta 0:00:01     |████████████████████▎           | 137.8MB 48.7MB/s eta 0:00:02��██▊           | 141.1MB 48.7MB/s eta 0:00:02
     |████████████████████████████████| 204kB 42.6MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [6]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [7]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In [15]:
rdd = sc.parallelize(range(100))

In [16]:
sum=rdd.sum()
n=rdd.count()
mean=sum/n
print(mean)

49.5


In [17]:
sortedAndIndexed=rdd.sortBy(lambda x:x).zipWithIndex()
n=sortedAndIndexed.count()
if n%2==1:
    index=(n-1)/2
    print(sortedAndIndexed.lookup(index))
else:
    index1=(n/2)-1
    index2=n/2
    value1=sortedAndIndexed.lookup(index1)
    value2=sortedAndIndexed.lookup(index2)
    print(value1)
    print(value2)

[49]
[50]


In [18]:
import math

In [19]:
sum=rdd.sum()
n=rdd.count()
mean=sum/n
print(mean)

49.5


In [20]:
sd=math.sqrt(rdd.map(lambda x: pow(x-mean,2)).sum()/n)
sd

28.86607004772212

In [21]:
n=float(n)
skewness=1/n*rdd.map(lambda x:pow(x-mean,3)/pow(sd,3)).sum()
skewness

6.217248937900877e-17

In [22]:
n=float(n)
kurtosis=1/n*rdd.map(lambda x:pow(x-mean,4)/pow(sd,4)).sum()
kurtosis

1.7997599759976002

In [57]:
import random

In [58]:
rddX=sc.parallelize(random.sample(range(100),100))
rddY=sc.parallelize(random.sample(range(100),100))


In [59]:
meanX=rddX.sum()/rddX.count()
meanY=rddY.sum()/rddY.count()
print(meanX)
print(meanY)

49.5
49.5


In [60]:
rddXY=rddX.zip(rddY)
rddXY.take(10)

[(65, 64),
 (87, 88),
 (13, 90),
 (94, 15),
 (7, 57),
 (46, 53),
 (91, 19),
 (0, 51),
 (57, 30),
 (69, 59)]

In [61]:
covXY = rddXY.map(lambda x_y : (x_y[0] - meanX)*(x_y[1] - meanY)).sum()/rddXY.count()
covXY

2.92

In [62]:
from math import sqrt
n=rddXY.count()
sdX=sqrt(rddX.map(lambda x:pow(x-meanX,2)).sum()/n)
sdY=sqrt(rddY.map(lambda y:pow(y-meanY,2)).sum()/n)
print(sdX)
print(sdY)

28.86607004772212
28.86607004772212


In [63]:
corrXY=covXY/(sdX*sdY)
corrXY

0.003504350435043504

In [65]:
import random
from pyspark.mllib.stat import Statistics

In [82]:
column1=sc.parallelize(range(100))
column2=sc.parallelize(range(100,200))
column3=sc.parallelize(list(reversed(range(100))))
column4=sc.parallelize(random.sample(range(100),100))
data=column1.zip(column2).zip(column3).zip(column4).map(lambda nested : (nested[0][0][0],nested[0][0][1],nested[0][1],nested[1])).map(lambda z:[z])
data.take(10)

[[(0, 100, 99, 31)],
 [(1, 101, 98, 82)],
 [(2, 102, 97, 68)],
 [(3, 103, 96, 25)],
 [(4, 104, 95, 12)],
 [(5, 105, 94, 22)],
 [(6, 106, 93, 38)],
 [(7, 107, 92, 54)],
 [(8, 108, 91, 85)],
 [(9, 109, 90, 78)]]

In [83]:
Statistics.corr(data)

array([[ 1.        ,  1.        , -1.        , -0.13269727],
       [ 1.        ,  1.        , -1.        , -0.13269727],
       [-1.        , -1.        ,  1.        ,  0.13269727],
       [-0.13269727, -0.13269727,  0.13269727,  1.        ]])

In [84]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

--2020-07-05 23:59:24--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2020-07-05 23:59:24--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2020-07-05 23:59:24--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Saving 

In [85]:
df.show()
df.printSchema()

+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth|
| 18| 49| 34|Accelerometer-201...|Brush_teeth|
| 19| 48| 34|Accelerometer-201...|Brush_teeth|
| 16| 53| 34|Accelerometer-201...|Brush_teeth|
| 18| 52| 35|

In [86]:
spark.sql('select class,count(*) from df group by class').show()

+--------------+--------+
|         class|count(1)|
+--------------+--------+
| Use_telephone|   15225|
| Standup_chair|   25417|
|      Eat_meat|   31236|
|     Getup_bed|   45801|
|   Drink_glass|   42792|
|    Pour_water|   41673|
|     Comb_hair|   23504|
|          Walk|   92254|
|  Climb_stairs|   40258|
| Sitdown_chair|   25036|
|   Liedown_bed|   11446|
|Descend_stairs|   15375|
|   Brush_teeth|   29829|
|      Eat_soup|    6683|
+--------------+--------+



In [87]:
df.groupBy('class').count().show()

+--------------+-----+
|         class|count|
+--------------+-----+
| Use_telephone|15225|
| Standup_chair|25417|
|      Eat_meat|31236|
|     Getup_bed|45801|
|   Drink_glass|42792|
|    Pour_water|41673|
|     Comb_hair|23504|
|          Walk|92254|
|  Climb_stairs|40258|
| Sitdown_chair|25036|
|   Liedown_bed|11446|
|Descend_stairs|15375|
|   Brush_teeth|29829|
|      Eat_soup| 6683|
+--------------+-----+



In [88]:
import pixiedust
from pyspark.sql.functions import col
counts = df.groupBy('class').count().orderBy('count')
display(counts)

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table SPARK_PACKAGES created successfully
Table USER_PREFERENCES created successfully
Table service_connections created successfully


DataFrame[class: string, count: bigint]

In [89]:
spark.sql('''
    select 
        *,
        max/min as minmaxratio -- compute minmaxratio based on previously computed values
        from (
            select 
                min(ct) as min, -- compute minimum value of all classes
                max(ct) as max, -- compute maximum value of all classes
                mean(ct) as mean, -- compute mean between all classes
                stddev(ct) as stddev -- compute standard deviation between all classes
                from (
                    select
                        count(*) as ct -- count the number of rows per class and rename it to ct
                        from df -- access the temporary query table called df backed by DataFrame df
                        group by class -- aggrecate over class
                )
        )   
''').show()

+----+-----+------------------+------------------+-----------------+
| min|  max|              mean|            stddev|      minmaxratio|
+----+-----+------------------+------------------+-----------------+
|6683|92254|31894.928571428572|21284.893716741157|13.80427951518779|
+----+-----+------------------+------------------+-----------------+



In [90]:
from pyspark.sql.functions import col, min, max, mean, stddev

df \
    .groupBy('class') \
    .count() \
    .select([ 
        min(col("count")).alias('min'), 
        max(col("count")).alias('max'), 
        mean(col("count")).alias('mean'), 
        stddev(col("count")).alias('stddev') 
    ]) \
    .select([
        col('*'),
        (col("max") / col("min")).alias('minmaxratio')
    ]) \
    .show()

+----+-----+------------------+------------------+-----------------+
| min|  max|              mean|            stddev|      minmaxratio|
+----+-----+------------------+------------------+-----------------+
|6683|92254|31894.928571428572|21284.893716741157|13.80427951518779|
+----+-----+------------------+------------------+-----------------+



In [103]:
spark.sql('select class,count(*) as count from df group by class order by count').show()

+--------------+-----+
|         class|count|
+--------------+-----+
|      Eat_soup| 6683|
|   Liedown_bed|11446|
| Use_telephone|15225|
|Descend_stairs|15375|
|     Comb_hair|23504|
| Sitdown_chair|25036|
| Standup_chair|25417|
|   Brush_teeth|29829|
|      Eat_meat|31236|
|  Climb_stairs|40258|
|    Pour_water|41673|
|   Drink_glass|42792|
|     Getup_bed|45801|
|          Walk|92254|
+--------------+-----+



In [104]:
display(counts)

DataFrame[class: string, count: bigint]